# Graph-Learning-Based Recommender System on MovieLens

### Group 9

- AGARWAL, Sahil
- WEI, Yuanjing
- ZHANG, Yujun yzhanglo@connect.ust.hk

Group project of COMP4222@HKUST in 2022 Fall.

# 1 Environment Configuration

In [ ]:
# change the path in the following
try:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/4222Group9'
except:
    %cd '/data/yzhanglo/4222project'

import comp4222
import recommenders
%pwd
%ls

/data/yzhanglo/4222project
backup_main.ipynb  LICENSE        ml-100k/          params.csv
comp4222/          LightGCN/      ml-latest-small/  recommenders/
cuda101.yml        lightgcn.yaml  movielens.ipynb   requirements.txt
KGAT_folder/       main.ipynb     movielens.zip


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from tensorboardX import SummaryWriter

import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

# easier to print by putting variable as a single line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# make matplotlib figures appear inline in the notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.2f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

/data/yzhanglo/miniconda/envs/cu101/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-22 11:41:33.947983: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
# Testing the module functionality
from comp4222 import b
comp4222.b.ok()

hahaha


# 2 MovieLens


We're using ml-latest-small from MovieLens. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018. The readme.md is avaliable [here](https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html).

## Data Loading for ml-latest-small

In [ ]:
# Download MovieLens data.
dataset_name = "ml-latest-small"
from urllib.request import urlretrieve
import zipfile
urlretrieve(f"https://files.grouplens.org/datasets/movielens/{dataset_name}.zip", "movielens.zip")
zipfile.ZipFile("movielens.zip", "r").extractall()

In [ ]:
movies = pd.read_csv(f"{dataset_name}/movies.csv")
genre_cols = [
    "(no genres listed)", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies

In [ ]:
tags = pd.read_csv(f"{dataset_name}/tags.csv")
tags

In [ ]:
ratings = pd.read_csv(f"{dataset_name}/ratings.csv")
ratings

# 5 Backup Models Definition

In [ ]:
from recommenders.utils.timer import Timer
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.utils.constants import (
    COL_DICT,
    DEFAULT_K,
    DEFAULT_USER_COL,
    DEFAULT_ITEM_COL,
    DEFAULT_RATING_COL,
    DEFAULT_PREDICTION_COL,
    DEFAULT_TIMESTAMP_COL,
    SEED,
)

# Helpers
import os
from tempfile import TemporaryDirectory
tmp_dir = TemporaryDirectory()
TRAIN_FILE = os.path.join(tmp_dir.name, "df_train.csv")
TEST_FILE = os.path.join(tmp_dir.name, "df_test.csv")

In [ ]:
from recommenders.utils.spark_utils import start_or_get_spark
spark = start_or_get_spark("PySpark", memory="32g")
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split

# fix random seeds to make sure out runs are reproducible
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

### ALS

In [ ]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType
from pyspark.ml.recommendation import ALS


def prepare_training_als(train, test):
    schema = StructType(
        (
            StructField(DEFAULT_USER_COL, IntegerType()),
            StructField(DEFAULT_ITEM_COL, IntegerType()),
            StructField(DEFAULT_RATING_COL, FloatType()),
            StructField(DEFAULT_TIMESTAMP_COL, LongType()),
        )
    )
    spark = start_or_get_spark()
    return spark.createDataFrame(train, schema).cache()

def prepare_metrics_als(train, test):
    schema = StructType(
        (
            StructField(DEFAULT_USER_COL, IntegerType()),
            StructField(DEFAULT_ITEM_COL, IntegerType()),
            StructField(DEFAULT_RATING_COL, FloatType()),
            StructField(DEFAULT_TIMESTAMP_COL, LongType()),
        )
    )
    spark = start_or_get_spark()
    return spark.createDataFrame(train, schema).cache(), spark.createDataFrame(test, schema).cache()

def predict_als(model, test):
    with Timer() as t:
        preds = model.transform(test)
    return preds, t

def train_als(params, data):
    symbol = ALS(**params)
    with Timer() as t:
        model = symbol.fit(data)
    return model, t

def recommend_k_als(model, test, train, top_k=DEFAULT_K, remove_seen=True):
    with Timer() as t:
        # Get the cross join of all user-item pairs and score them.
        users = train.select(DEFAULT_USER_COL).distinct()
        items = train.select(DEFAULT_ITEM_COL).distinct()
        user_item = users.crossJoin(items)
        dfs_pred = model.transform(user_item)

        # Remove seen items
        dfs_pred_exclude_train = dfs_pred.alias("pred").join(
            train.alias("train"),
            (dfs_pred[DEFAULT_USER_COL] == train[DEFAULT_USER_COL])
            & (dfs_pred[DEFAULT_ITEM_COL] == train[DEFAULT_ITEM_COL]),
            how="outer",
        )
        topk_scores = dfs_pred_exclude_train.filter(
            dfs_pred_exclude_train["train." + DEFAULT_RATING_COL].isNull()
        ).select(
            "pred." + DEFAULT_USER_COL,
            "pred." + DEFAULT_ITEM_COL,
            "pred." + DEFAULT_PREDICTION_COL,
        )
    return topk_scores, t


als_params = {
    "rank": 10,
    "maxIter": 20,
    "implicitPrefs": False,
    "alpha": 0.1,
    "regParam": 0.05,
    "coldStartStrategy": "drop",
    "nonnegative": False,
    "userCol": DEFAULT_USER_COL,
    "itemCol": DEFAULT_ITEM_COL,
    "ratingCol": DEFAULT_RATING_COL,
}

### NCF

In [ ]:
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset

def prepare_training_ncf(df_train, df_test):
    #df_train.sort_values(["userID"], axis=0, ascending=[True], inplace=True)
    #df_test.sort_values(["userID"], axis=0, ascending=[True], inplace=True)
    train = df_train.sort_values(["userID"], axis=0, ascending=[True])
    test = df_test.sort_values(["userID"], axis=0, ascending=[True])
    test = test[df_test["userID"].isin(train["userID"].unique())]
    test = test[test["itemID"].isin(train["itemID"].unique())]
    train.to_csv(TRAIN_FILE, index=False)
    test.to_csv(TEST_FILE, index=False)
    return NCFDataset(
        train_file=TRAIN_FILE,
        col_user=DEFAULT_USER_COL,
        col_item=DEFAULT_ITEM_COL,
        col_rating=DEFAULT_RATING_COL,
        seed=SEED,
    )


def train_ncf(params, data):
    model = NCF(n_users=data.n_users, n_items=data.n_items, **params)
    with Timer() as t:
        model.fit(data)
    return model, t


def recommend_k_ncf(model, test, train, top_k=DEFAULT_K, remove_seen=True):
    with Timer() as t:
        users, items, preds = [], [], []
        item = list(train[DEFAULT_ITEM_COL].unique())
        for user in train[DEFAULT_USER_COL].unique():
            user = [user] * len(item)
            users.extend(user)
            items.extend(item)
            preds.extend(list(model.predict(user, item, is_list=True)))
        topk_scores = pd.DataFrame(
            data={
                DEFAULT_USER_COL: users,
                DEFAULT_ITEM_COL: items,
                DEFAULT_PREDICTION_COL: preds,
            }
        )
        merged = pd.merge(
            train, topk_scores, on=[DEFAULT_USER_COL, DEFAULT_ITEM_COL], how="outer"
        )
        topk_scores = merged[merged[DEFAULT_RATING_COL].isnull()].drop(
            DEFAULT_RATING_COL, axis=1
        )
    # Remove temp files
    return topk_scores, t

ncf_params = {
    "model_type": "NeuMF",
    "n_factors": 4,
    "layer_sizes": [16, 8, 4],
    "n_epochs": 20,
    "batch_size": 1024,
    "learning_rate": 1e-3,
    "verbose": 10
}

### LightGCN

In [ ]:
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
    
def prepare_training_lightgcn(train, test):
    return ImplicitCF(train=train, test=test)

def train_lightgcn(params, data):
    hparams = prepare_hparams(**params)
    model = LightGCN(hparams, data)
    with Timer() as t:
        model.fit()
    return model, t

def recommend_k_lightgcn(model, test, train, top_k=DEFAULT_K, remove_seen=True):
    with Timer() as t:
        topk_scores = model.recommend_k_items(
            test, top_k=top_k, remove_seen=remove_seen
        )
    return topk_scores, t

lightgcn_param = {
    #"yaml_file": os.path.join("drive", "MyDrive", "4222Group9", "lightgcn.yaml"),
    "yaml_file": "lightgcn.yaml",
    "n_layers": 3,
    "batch_size": 1024,
    "epochs": 10,
    "learning_rate": 0.005,
    "eval_epoch": 5,
    "top_k": DEFAULT_K,
    "stacking_func": 1, #0: original, 1: exponential decay, 1.5: exponential increase, 2: trainable with random initialization, 3: trainable with unified initialization
}

### KGAT

In [ ]:
import sys
import random
from time import time
import os

import pandas as pd
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

from KGAT_folder.KGAT import KGAT
from KGAT_folder.log_helper import *
from KGAT_folder.parser_kgat import *
from KGAT_folder.metrics import *
from KGAT_folder.model_helper import *
from KGAT_folder.loader_kgat import DataLoaderKGAT

def train_kgat():
    args = parse_kgat_args()
    time0 = time()
    model, data, Ks, device = train(args)
    time1 = time()
    t = time1-time0
    return model, data, Ks, device, t

def evaluate_kgat(model, dataloader, Ks, device):
    test_batch_size = dataloader.test_batch_size
    train_user_dict = dataloader.train_user_dict
    test_user_dict = dataloader.test_user_dict

    model.eval()

    user_ids = list(test_user_dict.keys())
    user_ids_batches = [user_ids[i: i + test_batch_size] for i in range(0, len(user_ids), test_batch_size)]
    user_ids_batches = [torch.LongTensor(d) for d in user_ids_batches]

    n_items = dataloader.n_items
    item_ids = torch.arange(n_items, dtype=torch.long).to(device)
    
    cf_scores = []
    metric_names = ['precision', 'recall', 'ndcg']
    metrics_dict = {k: {m: [] for m in metric_names} for k in Ks}

    with tqdm(total=len(user_ids_batches), desc='Evaluating Iteration') as pbar:
        for batch_user_ids in user_ids_batches:
            batch_user_ids = batch_user_ids.to(device)

            with torch.no_grad():
                batch_scores = model(batch_user_ids, item_ids, mode='predict')       # (n_batch_users, n_items)

            batch_scores = batch_scores.cpu()
            batch_metrics = calc_metrics_at_k(batch_scores, train_user_dict, test_user_dict, batch_user_ids.cpu().numpy(), item_ids.cpu().numpy(), Ks)

            cf_scores.append(batch_scores.numpy())
            for k in Ks:
                for m in metric_names:
                    metrics_dict[k][m].append(batch_metrics[k][m])
            pbar.update(1)

    cf_scores = np.concatenate(cf_scores, axis=0)

    for k in Ks:
        for m in metric_names:
            metrics_dict[k][m] = np.concatenate(metrics_dict[k][m]).mean()
    return cf_scores, metrics_dict

    def train(args):
    # seed
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    data = DataLoaderKGAT(args, logging)
    if args.use_pretrain == 1:
        user_pre_embed = torch.tensor(data.user_pre_embed)
        item_pre_embed = torch.tensor(data.item_pre_embed)
    else:
        user_pre_embed, item_pre_embed = None, None
    model = KGAT(args, data.n_users, data.n_entities, data.n_relations, data.A_in, user_pre_embed, item_pre_embed)
    model.to(device)

    best_epochs = -1 # Can change the epochs here

    ref_epochs = 10
    

    cf_optimizer = optim.Adam(model.parameters(), lr=args.lr)
    kg_optimizer = optim.Adam(model.parameters(), lr=args.lr)

    best_recalls = 0 # The optimiser will affect the values of the recall

    ref_recalls = 50

    Ks = eval(args.Ks)
    k_min = min(Ks)
    k_max = max(Ks)

    epoch_list = []
    metrics_list = {k: {'precision': [], 'recall': [], 'ndcg': []} for k in Ks}

    for epoch in range(1, args.n_epoch + 1):
        time0 = time()
        model.train()
        time1 = time()
        cf_total_loss = 0
        n_cf_batch = data.n_cf_train // data.cf_batch_size + 1

        for iter in range(1, n_cf_batch + 1):
            time2 = time()
            cf_batch_user, cf_batch_pos_item, cf_batch_neg_item = data.generate_cf_batch(data.train_user_dict, data.cf_batch_size)
            cf_batch_user = cf_batch_user.to(device)
            cf_batch_pos_item = cf_batch_pos_item.to(device)
            cf_batch_neg_item = cf_batch_neg_item.to(device)
            cf_batch_loss = model(cf_batch_user, cf_batch_pos_item, cf_batch_neg_item, mode='train_cf')

            if np.isnan(cf_batch_loss.cpu().detach().numpy()):
                logging.info('ERROR (CF Training): Epoch {:04d} Iter {:04d} / {:04d} Loss is nan.'.format(epoch, iter, n_cf_batch))
                sys.exit()

            cf_batch_loss.backward()
            cf_optimizer.step()
            cf_optimizer.zero_grad()
            cf_total_loss += cf_batch_loss.item()

            if (iter % args.cf_print_every) == 0:
                logging.info('CF Training: Epoch {:04d} Iter {:04d} / {:04d} | Time {:.1f}s | Iter Loss {:.4f} | Iter Mean Loss {:.4f}'.format(epoch, iter, n_cf_batch, time() - time2, cf_batch_loss.item(), cf_total_loss / iter))
        logging.info('CF Training: Epoch {:04d} Total Iter {:04d} | Total Time {:.1f}s | Iter Mean Loss {:.4f}'.format(epoch, n_cf_batch, time() - time1, cf_total_loss / n_cf_batch))

        time3 = time()
        kg_total_loss = 0
        n_kg_batch = data.n_kg_train // data.kg_batch_size + 1

        for iter in range(1, n_kg_batch + 1):
            time4 = time()
            kg_batch_head, kg_batch_relation, kg_batch_pos_tail, kg_batch_neg_tail = data.generate_kg_batch(data.train_kg_dict, data.kg_batch_size, data.n_users_entities)
            kg_batch_head = kg_batch_head.to(device)
            kg_batch_relation = kg_batch_relation.to(device)
            kg_batch_pos_tail = kg_batch_pos_tail.to(device)
            kg_batch_neg_tail = kg_batch_neg_tail.to(device)

            kg_batch_loss = model(kg_batch_head, kg_batch_relation, kg_batch_pos_tail, kg_batch_neg_tail, mode='train_kg')

            if np.isnan(kg_batch_loss.cpu().detach().numpy()):
                logging.info('ERROR (KG Training): Epoch {:04d} Iter {:04d} / {:04d} Loss is nan.'.format(epoch, iter, n_kg_batch))
                sys.exit()

            kg_batch_loss.backward()
            kg_optimizer.step()
            kg_optimizer.zero_grad()
            kg_total_loss += kg_batch_loss.item()

            if (iter % args.kg_print_every) == 0:
                logging.info('KG Training: Epoch {:04d} Iter {:04d} / {:04d} | Time {:.1f}s | Iter Loss {:.4f} | Iter Mean Loss {:.4f}'.format(epoch, iter, n_kg_batch, time() - time4, kg_batch_loss.item(), kg_total_loss / iter))
        logging.info('KG Training: Epoch {:04d} Total Iter {:04d} | Total Time {:.1f}s | Iter Mean Loss {:.4f}'.format(epoch, n_kg_batch, time() - time3, kg_total_loss / n_kg_batch))

        time5 = time()
        h_list = data.h_list.to(device)
        t_list = data.t_list.to(device)
        r_list = data.r_list.to(device)
        relations = list(data.laplacian_dict.keys())
        model(h_list, t_list, r_list, relations, mode='update_att')
        logging.info('Update Attention: Epoch {:04d} | Total Time {:.1f}s'.format(epoch, time() - time5))

        logging.info('CF + KG Training: Epoch {:04d} | Total Time {:.1f}s'.format(epoch, time() - time0))
    return model, data, Ks, device




# Prepare for training

In [ ]:


from recommenders.evaluation.python_evaluation import (
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
)
def ranking_metrics_python(test, predictions, k=DEFAULT_K):
    return {
        "MAP": map_at_k(test, predictions, k=k, **COL_DICT),
        "nDCG@k": ndcg_at_k(test, predictions, k=k, **COL_DICT),
        "Precision@k": precision_at_k(test, predictions, k=k, **COL_DICT),
        "Recall@k": recall_at_k(test, predictions, k=k, **COL_DICT),
    }

#params = {
#    "als": als_params,
#    "ncf": ncf_params,
#    "lightgcn": lightgcn_param,
#}
prepare_training_data = {
    "als": prepare_training_als,
    "ncf": prepare_training_ncf,
    "lightgcn": prepare_training_lightgcn,
}


from recommenders.evaluation.spark_evaluation import (
    SparkRatingEvaluation,
    SparkRankingEvaluation,
)
def rating_metrics_pyspark(test, predictions):
    rating_eval = SparkRatingEvaluation(test, predictions, **COL_DICT)
    return {
        "RMSE": rating_eval.rmse(),
        "MAE": rating_eval.mae(),
        "R2": rating_eval.exp_var(),
        "Explained Variance": rating_eval.rsquared(),
    }
def ranking_metrics_pyspark(test, predictions, k=DEFAULT_K):
    rank_eval = SparkRankingEvaluation(
        test, predictions, k=k, relevancy_method="top_k", **COL_DICT
    )
    return {
        "MAP": rank_eval.map_at_k(),
        "nDCG@k": rank_eval.ndcg_at_k(),
        "Precision@k": rank_eval.precision_at_k(),
        "Recall@k": rank_eval.recall_at_k(),
    }

trainer = {
    "als": lambda params, data: train_als(params, data),
    "ncf": lambda params, data: train_ncf(params, data),
    "lightgcn": lambda params, data: train_lightgcn(params, data),
}
ranking_predictor = {
    "als": lambda model, test, train: recommend_k_als(model, test, train),
    "ncf": lambda model, test, train: recommend_k_ncf(model, test, train),
    "lightgcn": lambda model, test, train: recommend_k_lightgcn(model, test, train),
}
ranking_evaluator = {
    "als": lambda test, predictions, k: ranking_metrics_pyspark(test, predictions, k),
    "ncf": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "lightgcn": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
}
metrics = {
    #"als": ["rating", "ranking"],
    "als" : ["ranking"],
    "ncf": ["ranking"],
    "lightgcn": ["ranking"]
}

In [ ]:
from recommenders.evaluation.spark_evaluation import (
    SparkRatingEvaluation,
    SparkRankingEvaluation,
)
def rating_metrics_pyspark(test, predictions):
    rating_eval = SparkRatingEvaluation(test, predictions, **COL_DICT)
    return {
        "RMSE": rating_eval.rmse(),
        "MAE": rating_eval.mae(),
        "R2": rating_eval.exp_var(),
        "Explained Variance": rating_eval.rsquared(),
    }
def ranking_metrics_pyspark(test, predictions, k=DEFAULT_K):
    rank_eval = SparkRankingEvaluation(
        test, predictions, k=k, relevancy_method="top_k", **COL_DICT
    )
    return {
        "MAP": rank_eval.map_at_k(),
        "nDCG@k": rank_eval.ndcg_at_k(),
        "Precision@k": rank_eval.precision_at_k(),
        "Recall@k": rank_eval.recall_at_k(),
    }
trainer = {
    "lightgcn": lambda params, data: train_lightgcn(params, data),
}
ranking_predictor = {
    "lightgcn": lambda model, test, train: recommend_k_lightgcn(model, test, train),
}
ranking_evaluator = {
    "lightgcn": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
}
metrics = {
    "lightgcn": ["ranking"]
}

## Sanity Check by Overfitting on Small Data

In [ ]:
#!pip install pytorch-lightning
#from pytorch_lightning import Trainer, seed_everything
#from LightGCN.code.main_lgcn import sanity_check

#seed_everything(42, workers=True)

#model = sanity_check()
#trainer = Trainer(max_epochs=10000, overfit_batches=0.01)
#trainer.fit(model)

# 6 Hyperparameter Tunning

In [ ]:
def generate_summary(data, algo, k, train_time, time_rating, rating_metrics, time_ranking, ranking_metrics):
    summary = {"Data": data, "K": k, "Train time (s)": train_time, "Recommending time (s)": time_ranking}
    if rating_metrics is None:
        rating_metrics = {
            "RMSE": np.nan,
            "MAE": np.nan,
            "R2": np.nan,
            "Explained Variance": np.nan,
        }
    if ranking_metrics is None:
        ranking_metrics = {
            "MAP": np.nan,
            "nDCG@k": np.nan,
            "Precision@k": np.nan,
            "Recall@k": np.nan,
        }
    summary.update(ranking_metrics)
    return summary

In [ ]:
data_sizes = ["100k"] # Movielens data size: 100k, 1m, 10m, or 20m
algorithms = [ "lightgcn"]

lightgcn_param_dict = {
    #"yaml_file": os.path.join("drive", "MyDrive", "4222Group9", "lightgcn.yaml"),
    "yaml_file": "lightgcn.yaml",
    "embed_size" : [32, 64, 128], # the embedding dimension of users and items
    # "n_layers": 3, # number of layers of the model

    # "batch_size": 1024,
    "decay" : 0.0001, # l2 regularization for embedding parameters
    "epochs": 200,
    "learning_rate": [0.0005, 0.001, 0.002],
    "eval_epoch": 50,
    # "top_k": DEFAULT_K,
    "stacking_func": [0,1,1.5], #0: original, 1: exponential decay, 1.5: exponential increase, 2: trainable with random initialization, 3: trainable with unified initialization

    # "save_model" : False,
    # "save_epoch" : 10,
    "save_board": True,
    "board_comment": "",
}

In [ ]:
from recommenders.tuning.parameter_sweep import generate_param_grid
lightgcn_params = generate_param_grid(lightgcn_param_dict)

### Training

In [ ]:
import traceback
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split

In [ ]:
%%time
data_sizes = ["100k"] # Movielens data size: 100k, 1m, 10m, or 20m
algorithms = [ "lightgcn"]

# For each data size and each algorithm, a recommender is evaluated. 
cols = ["Data", "Algo", "K", "Train time (s)", "Predicting time (s)",  "Recommending time (s)", "MAP", "nDCG@k", "Precision@k", "Recall@k"]
df_results = pd.DataFrame(columns=cols)

for data_size in data_sizes:

    # Load the dataset
    #df = movielens.load_pandas_df(
    #    size=data_size,
    #    header=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL]
    #)
    if data_size=='100k':
        df = pd.read_csv('ml-100k/u.data', sep='\t', header=None)

    df.columns = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL]
    df[DEFAULT_RATING_COL] = df[DEFAULT_RATING_COL].astype(float)
    print("Size of Movielens {}: {}".format(data_size, df.shape))
    
    # Split the dataset
    df_train, df_test = python_stratified_split(df,
                                                ratio=0.75, 
                                                min_rating=1, 
                                                filter_by="item", 
                                                col_user=DEFAULT_USER_COL, 
                                                col_item=DEFAULT_ITEM_COL,
                                                )

    #df_train = "../LightGCN/data/movielens/train.txt"
    # Loop through the algos
    for algo in algorithms:
        print(f"\nComputing {algo} algorithm on Movielens {data_size}")
        if algo == 'kgat':
            model, data, Ks, device, time_train = train_kgat()
            _, metrics_dict_kgat = evaluate_kgat(model, data, Ks, device)
            print(metrics_dict_kgat)
            # Record results
            #summary = generate_summary('100k', algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
            #df_results.loc[df_results.shape[0] + 1] = summary
            
        else:
            # Data prep for training set
            obj = prepare_training_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
            
            for model_params in lightgcn_params:
                print("training using the params:", model_params)
                model_params["board_comment"] = str(model_params)

                # Train the model
                #try:
                model, time_train = trainer[algo](model_params, obj)
                #except:
                    #traceback.print_exc()
                
                print(f"Training time: {time_train}s")
                        
                # Predict and evaluate
                #train, test = prepare_metrics_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
                train, test = df_train, df_test
                
                if "rating" in metrics[algo]:   
                    pass
                else:
                    ratings = None
                    time_rating = np.nan
                
                if "ranking" in metrics[algo]:
                    # Predict for ranking
                    top_k_scores, time_ranking = ranking_predictor[algo](model, test, train)
                    print(f"Ranking prediction time: {time_ranking}s")
                    
                    # Evaluate for rating
                    rankings = ranking_evaluator[algo](test, top_k_scores, DEFAULT_K)
                else:
                    rankings = None
                    time_ranking = np.nan
                    
                # Record results
                summary = generate_summary(data_size, algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
                df_results.loc[df_results.shape[0] + 1] = summary
        
print("\nComputation finished")

## Training Result Plot

Check the result table, and then filter on that table to see suitable params setting

In [6]:
df_result = df_results
df_result['param'] = lightgcn_params
with pd.option_context('display.max_rows', None):
    with pd.option_context('display.max_colwidth', None):
        df_result.to_csv('params.csv')
        #from IPython.display import HTML
        #HTML(df_result.to_html())
        df_result

NameError: name 'df_results' is not defined

In [10]:
df_result = pd.read_csv('params.csv')
df_result

,Unnamed: 0,Data,Algo,K,Train time (s),Predicting time (s),Recommending time (s),MAP,nDCG@k,Precision@k,Recall@k,param
0,1,100k,lightgcn,10,148.77,NaN,0.10,0.12,0.41,0.35,0.19,"{'embed_size': 32, 'learning_rate': 0.0005, 's..."
1,2,100k,lightgcn,10,152.58,NaN,0.12,0.13,0.44,0.38,0.21,"{'embed_size': 32, 'learning_rate': 0.0005, 's..."
2,3,100k,lightgcn,10,154.85,NaN,0.09,0.06,0.22,0.20,0.12,"{'embed_size': 32, 'learning_rate': 0.0005, 's..."
3,4,100k,lightgcn,10,150.40,NaN,0.13,0.13,0.44,0.38,0.21,"{'embed_size': 32, 'learning_rate': 0.001, 'st..."
4,5,100k,lightgcn,10,154.14,NaN,0.09,0.14,0.45,0.39,0.21,"{'embed_size': 32, 'learning_rate': 0.001, 'st..."
...,...,...,...,...,...,...,...,...,...,...,...,...
22,23,100k,lightgcn,10,166.56,NaN,0.11,0.13,0.43,0.38,0.21,"{'embed_size': 128, 'learning_rate': 0.001, 's..."
23,24,100k,lightgcn,10,174.41,NaN,0.11,0.06,0.23,0.21,0.12,"{'embed_size': 128, 'learning_rate': 0.001, 's..."
24,25,100k,lightgcn,10,169.75,NaN,0.12,0.13,0.44,0.39,0.22,"{'embed_size': 128, 'learning_rate': 0.002, 's..."
25,26,100k,lightgcn,10,168.98,NaN,0.12,0.10,0.35,0.31,0.17,"{'embed_size': 128, 'learning_rate': 0.002, 's..."


In [16]:
for key, value in eval(df_result.at[0, "param"]).items():
    for i in np.arange(df_result.shape[0]):
        df_result.at[i, key] = eval(df_result.at[i, "param"])[key]
df_result

,Unnamed: 0,Data,Algo,K,Train time (s),Predicting time (s),Recommending time (s),MAP,nDCG@k,Precision@k,Recall@k,param,embed_size,learning_rate,stacking_func,yaml_file,decay,epochs,eval_epoch
0,1,100k,lightgcn,10,148.77,NaN,0.10,0.12,0.41,0.35,0.19,"{'embed_size': 32, 'learning_rate': 0.0005, 's...",32.00,0.00,0.00,lightgcn.yaml,0.00,200.00,50.00
1,2,100k,lightgcn,10,152.58,NaN,0.12,0.13,0.44,0.38,0.21,"{'embed_size': 32, 'learning_rate': 0.0005, 's...",32.00,0.00,1.00,lightgcn.yaml,0.00,200.00,50.00
2,3,100k,lightgcn,10,154.85,NaN,0.09,0.06,0.22,0.20,0.12,"{'embed_size': 32, 'learning_rate': 0.0005, 's...",32.00,0.00,1.50,lightgcn.yaml,0.00,200.00,50.00
3,4,100k,lightgcn,10,150.40,NaN,0.13,0.13,0.44,0.38,0.21,"{'embed_size': 32, 'learning_rate': 0.001, 'st...",32.00,0.00,0.00,lightgcn.yaml,0.00,200.00,50.00
4,5,100k,lightgcn,10,154.14,NaN,0.09,0.14,0.45,0.39,0.21,"{'embed_size': 32, 'learning_rate': 0.001, 'st...",32.00,0.00,1.00,lightgcn.yaml,0.00,200.00,50.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,23,100k,lightgcn,10,166.56,NaN,0.11,0.13,0.43,0.38,0.21,"{'embed_size': 128, 'learning_rate': 0.001, 's...",128.00,0.00,1.00,lightgcn.yaml,0.00,200.00,50.00
23,24,100k,lightgcn,10,174.41,NaN,0.11,0.06,0.23,0.21,0.12,"{'embed_size': 128, 'learning_rate': 0.001, 's...",128.00,0.00,1.50,lightgcn.yaml,0.00,200.00,50.00
24,25,100k,lightgcn,10,169.75,NaN,0.12,0.13,0.44,0.39,0.22,"{'embed_size': 128, 'learning_rate': 0.002, 's...",128.00,0.00,0.00,lightgcn.yaml,0.00,200.00,50.00
25,26,100k,lightgcn,10,168.98,NaN,0.12,0.10,0.35,0.31,0.17,"{'embed_size': 128, 'learning_rate': 0.002, 's...",128.00,0.00,1.00,lightgcn.yaml,0.00,200.00,50.00


In [32]:
import plotly
import plotly.graph_objs as go

recall = df_result['Recall@k']
print(recall.min(), recall.max())
recall = (recall - recall.min()) / (recall.max() - recall.min())

fig1 = go.Scatter3d(x=df_result['embed_size'],
                    y=df_result['learning_rate'],
                    z=df_result['stacking_func'],
                    text = [item for item in recall.astype(str)],
                    marker=dict(color = recall,
                                reversescale=False,
                                colorscale='Blues',
                                size=7),
                    line=dict (width=0.02),
                    mode='markers')

#Make Plot.ly Layout
mylayout = go.Layout(scene=dict(xaxis=dict(title="embed_size"),
                                yaxis=dict(title="learning_rate"),
                                zaxis=dict(title="stacking_func")),)

#Plot and save html
plotly.offline.plot({"data": [fig1],
                     "layout": mylayout},
                     auto_open=True)
# reference: https://medium.com/@prasadostwal/multi-dimension-plots-in-python-from-2d-to-6d-9a2bf7b8cc74

0.1175262640396166 0.2273533123813273


'temp-plot.html'

In [ ]:
lightgcn_params # see the corresponding param

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/board

Upload the satisfactory result of tensorboard(change the name fit into folder name):

In [ ]:
!tensorboard dev upload \
  --logdir logs/fit \
  --name "BiasedGCN experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot

# 7 Comparisons on Movielens and Movie

In [ ]:
data_sizes = ["100k","1m"] # Movielens data size: 100k, 1m, 10m, or 20m
#algorithms = [ "lightgcn"]
algorithms = ["als", "ncf", "lightgcn", "kgat"]

## Print the result summary

In [ ]:
df_results

# 8 Credit and Reference

1. https://github.com/microsoft/recommenders